In [1]:
from cocoon_data import *

In [2]:
# load embedded vocabulary file
reference_df = pd.read_csv('/home/zh2408/ohdsi/Procedure.csv')

In [3]:
index = load_embedding(reference_df, label_embedding='embedding')

In [4]:
procedure_df = pd.read_csv('./files/procedures.csv')

In [7]:
# build embedding for your file
procedure_df["label"] = procedure_df["DESCRIPTION"]
embed_labels(procedure_df, './files/embedded_procedures.csv')

Embedding Labels:   0%|          | 0/5 [00:00<?, ?label/s]

Embedding Labels: 100%|██████████| 5/5 [00:00<00:00,  6.08label/s]

All labels embedded and CSV updated.


,label,index_ids,CODE,DESCRIPTION,embedding
0,Admission to orthopedic department,[4],305428000,Admission to orthopedic department,"[0.027575980871915817, 0.008947665803134441, -..."
1,Clavicle X-ray,[3],168594001,Clavicle X-ray,"[0.003449691692367196, 0.00801424216479063, 0...."
2,Injection of tetanus antitoxin,[7],384700001,Injection of tetanus antitoxin,"[-0.03375847265124321, 0.015600106678903103, -..."
3,Medication Reconciliation (procedure),"[0, 1, 2, 8, 9]",430193006,Medication Reconciliation (procedure),"[0.02277933992445469, 0.02867008186876774, 0.0..."
4,Suture open wound,"[5, 6]",288086009,Suture open wound,"[0.0003375147352926433, 0.0011483834823593497,..."


In [10]:
# use the embedding to search for the nearest neighbor
procedure_synthea_embedded_label = pd.read_csv('./files/embedded_procedures.csv')
D, I = df_search(procedure_synthea_embedded_label, index)

In [11]:
# below are the matched results using embedding
display_matches(reference_df=reference_df, input_df=procedure_synthea_embedded_label, I=I)

HTML(value='<b>Input Label</b>: <table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: ri…

In [14]:
# relation-based entity matching
entity_relation_match(input_df = procedure_synthea_embedded_label, 
                        attributes = ["DESCRIPTION"], 
                        I = I, 
                        refernece_df = reference_df, 
                        label = "label")

procedure_synthea_embedded_label.to_csv('./files/procedure_synthea_matched.csv', index=False)

In [15]:
procedure_synthea_matched = pd.read_csv('./files/procedure_synthea_matched.csv')

In [16]:
display_entity_matches(procedure_synthea_matched)

HTML(value='<h1>Input Data</h1>\n<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: r…

In [17]:
write_report_to_file(procedure_synthea_matched, "./files/standardization_report.html")